# Data understanding e data preparation

# 1. Objetivo do notebook:

O objetivo desde notebook é fornecer informações primárias sobre o conjunto de dados 'cru' e realizar pré-processamentos para etapas de análise exploratória e modelagem

<u>Output</u>: Dados pré-processados (**'../data/processed/card_transactions_cbk_prepared.csv'**)

# 2. Inspeção dos dados

In [1]:
import pandas as pd
import numpy as np
import datetime
import holidays

In [2]:
df_aba1 = pd.read_excel('../data/raw/Missão_Stone_-_Dados_de_trx_(3).xlsx', engine='openpyxl', sheet_name='Aba 1')

## 2.1 Descrições

In [3]:
print ('Aba 1')
print ('------------------------------------------')
print ("Linhas     : " ,df_aba1.shape[0])
print ("Colunas    : " ,df_aba1.shape[1])
print ("\nFeatures : \n" ,df_aba1.columns.tolist())
print ("\nValores nulos :  ", df_aba1.isnull().sum().values.sum())
print ("\nValores únicos :  \n",df_aba1.nunique())

Aba 1
------------------------------------------
Linhas     :  11128
Colunas    :  5

Features : 
 ['Dia', 'Hora', 'Valor', 'Cartão', 'CBK']

Valores nulos :   124

Valores únicos :  
 Dia         154
Hora      10100
Valor       629
Cartão     9261
CBK           2
dtype: int64


In [4]:
for column in df_aba1.columns.tolist():
    print(df_aba1[column].value_counts())
    print('--------------------------------')

Dia
2015-05-11 00:00:00        841
2015-05-29 00:00:00        732
2015-05-15 00:00:00        659
2015-05-28 00:00:00        611
2015-05-12 00:00:00        581
                          ... 
2015-05-30 16:34:22.000      1
2015-05-30 16:25:48.000      1
2015-05-30 16:24:46.000      1
2015-05-30 16:21:38.000      1
2015-05-30 23:51:31.000      1
Name: count, Length: 154, dtype: int64
--------------------------------
Hora
15          11
20          10
99          10
10           7
70           7
            ..
22:12:10     1
22:12:25     1
22:13:06     1
22:13:21     1
33           1
Name: count, Length: 10100, dtype: int64
--------------------------------
Valor
154                 561
46                  372
99                  328
34.5                290
78.4                287
                   ... 
172                   1
129.6                 1
465                   1
58.78                 1
518759******0329      1
Name: count, Length: 629, dtype: int64
------------------------------

In [5]:
df_aba1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128 entries, 0 to 11127
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Dia     11128 non-null  object
 1   Hora    11128 non-null  object
 2   Valor   11128 non-null  object
 3   Cartão  11128 non-null  object
 4   CBK     11004 non-null  object
dtypes: object(5)
memory usage: 434.8+ KB


In [6]:
df_aba1.head()

,Dia,Hora,Valor,Cartão,CBK
0,2015-05-01 00:00:00,00:01:54,36.54,536518******2108,Não
1,2015-05-01 00:00:00,00:03:46,36.54,536518******2108,Não
2,2015-05-01 00:00:00,00:08:50,69,453211******1239,Não
3,2015-05-01 00:00:00,00:27:00,193.43,548827******1705,Não
4,2015-05-01 00:00:00,01:32:46,132,531681******9778,Não


In [7]:
df_aba1.tail()

,Dia,Hora,Valor,Cartão,CBK
11123,2015-05-30 23:07:01.000,53,514868******7409,Não,NaN
11124,2015-05-30 23:08:47.000,15,439354******5281,Não,NaN
11125,2015-05-30 23:15:24.000,20,549167******1648,Não,NaN
11126,2015-05-30 23:17:41.000,70,518759******8384,Não,NaN
11127,2015-05-30 23:51:31.000,20,518759******0329,Não,NaN


In [8]:
pd.DataFrame(100*df_aba1.isnull().sum(axis=0)).sort_values(by=0, ascending=False)/df_aba1.shape[0]

,0
CBK,1.114306
Dia,0.000000
Hora,0.000000
Valor,0.000000
Cartão,0.000000


### 2.1 Adequação dos dados e data cleaning

#### 2.1.1 Valores temporais

Percebe-se no value_counts que a coluna 'Hora' tem formatos diferentes e números incoerentes (como 33). Além disso, percebe-se uma melhoria unindo os dois em um único timestamp, assim:

In [9]:
def correct_hour_format(x):
    """
    Esta função tenta converter o valor de entrada `x` para um objeto `timedelta` do pandas. 
    Se a conversão falhar, a função assume que a entrada é uma hora inteira (0-23) e formata essa hora como 'HH:00:00'.

    Parâmetros:
    x(str ou int)->int: O valor de entrada representando a hora.

    Retorno:
    pandas.Timedelta -> Valor da hora convertido em um objeto `timedelta` do pandas.
    """
    try:
        hour = pd.to_timedelta(str(x))
    except: 
        hour = pd.to_timedelta(f'{int(x):02d}:00:00')
    return hour

In [10]:
# Correção dos dtypes
df_aba1['Hora'] = df_aba1['Hora'].apply(correct_hour_format)
df_aba1['Dia'] = pd.to_datetime(df_aba1['Dia'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

# Limpeza de anos 'absurdos' (muito no passado ou futuro)
current_year = datetime.datetime.now().year
df_aba1 = df_aba1[(df_aba1['Dia'].dt.year >= 1900) & (df_aba1['Dia'].dt.year <= current_year)]

df_aba1['transaction_timestamp'] = df_aba1['Dia'] + df_aba1['Hora']

#Limpeza de transaction_timestamps com horas fora do range aceitável (dentro das 24 horas)
df_aba1 = df_aba1[(df_aba1['transaction_timestamp'].dt.hour >= 0) & (df_aba1['transaction_timestamp'].dt.hour < 24)]

#### 2.1.2 Valores numéricos

Repara-se também pelo value_counts um código de cartão na coluna valor, então:

In [11]:
df_aba1 = df_aba1[pd.to_numeric(df_aba1['Valor'], errors='coerce').notnull()]

df_aba1['Valor'] = df_aba1['Valor'].astype('float')

#### 2.1.3 Target

Nulos em uma abordagem de crédito/fraude é uma zona cinzenta. Como visto acima, não haverá muita perda de dados com o drop (~1%), então opta-se por isso

In [12]:
df_aba1 = df_aba1.dropna(subset=['CBK'])
df_aba1['CBK'] = df_aba1['CBK'].map({'Não': 0, 'Sim': 1}).fillna(df_aba1['CBK'])

Agora, verificando tudo:

In [13]:
print(f'Range de datas: {df_aba1.Dia.min().date()} - {df_aba1.Dia.max().date()}')
print(f'Range de horas: {str(df_aba1.Hora.min()).replace("0 days", "")} - {str(df_aba1.Hora.max()).replace("0 days", "")}')


df_aba1[['Valor', 'CBK']].describe(percentiles = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]).T

Range de datas: 2015-05-01 - 2015-05-30
Range de horas:  00:00:00 -  23:59:44


,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
Valor,11004.0,130.023628,141.855348,1.0,10.0,27.0,55.0,99.0,154.0,250.0,660.0,2920.0
CBK,11004.0,0.051618,0.221264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [14]:
df_aba1.head(3)

,Dia,Hora,Valor,Cartão,CBK,transaction_timestamp
0,2015-05-01,0 days 00:01:54,36.54,536518******2108,0,2015-05-01 00:01:54
1,2015-05-01,0 days 00:03:46,36.54,536518******2108,0,2015-05-01 00:03:46
2,2015-05-01,0 days 00:08:50,69.00,453211******1239,0,2015-05-01 00:08:50


In [15]:
df_aba1.tail(3)

,Dia,Hora,Valor,Cartão,CBK,transaction_timestamp
11001,2015-05-30,0 days 23:15:24,20.0,549167******1648,0,2015-05-30 23:15:24
11002,2015-05-30,0 days 23:17:41,70.0,518759******8384,0,2015-05-30 23:17:41
11003,2015-05-30,0 days 23:51:31,20.0,518759******0329,0,2015-05-30 23:51:31


### 2.2 Feature Engineering

#### 2.2.1 Valores temporais

- Extração de day of the week, flag de feriado, hora do dia dos dados temporais, trimestre do ano

In [16]:
def periodo_do_dia(timestamp):
    """
    Avalia a hora do timestamp fornecido e classifica o período do dia em uma das seguintes categorias: 
    - 'madrugada' (entre 00:00:01 e 05:29:59)
    - 'dia' (entre 05:30:00 e 11:59:59)
    - 'tarde' (entre 12:00:00 e 17:59:59)
    - 'noite' (entre 18:00:00 e 00:00:00)

    Parâmetros:
    timestamp -> (datetime.datetime)

    Retorno:
    str
    """
    hora = timestamp.time()
    madrugada_inicio = datetime.time(0, 0, 1)
    manha_inicio = datetime.time(5, 30, 0)
    tarde_inicio = datetime.time(12, 0, 0)
    noite_inicio = datetime.time(18, 0, 0)
    if madrugada_inicio <= hora < manha_inicio:
        return 'madrugada'
    elif manha_inicio <= hora < tarde_inicio:
        return 'dia'
    elif tarde_inicio <= hora < noite_inicio:
        return 'tarde'
    else:
        return 'noite'

In [17]:
df_aba1['periodo_do_dia'] = df_aba1['transaction_timestamp'].apply(periodo_do_dia)

df_aba1['dia_da_semana'] = pd.to_datetime(df_aba1['transaction_timestamp']).dt.day_name(locale='pt_BR')

df_aba1['flag_feriado'] = df_aba1['transaction_timestamp'].dt.date.apply(lambda x: 1 if x in holidays.Brazil() else 0)

# df_aba1['trimestre'] = df_aba1['transaction_timestamp'].dt.quarter -> No caso, só há um trimestre pois só há dados de maio

#### 2.2.2 Valor da transação

Opta-se por não alterar pois, caso o algoritmo escolhido seja uma árvore de decisão, ele é capaz de fazer os próprios cortes em valores numéricos.

#### 2.2.3 Cartão

Após uma pesquisa, descobre-se que o primeiro digito de um cartão corresponde à bandeira e os 5 próximos dígitos, a instituição bancária. Assim:

In [18]:
df_aba1['bandeira_cartao'] = df_aba1['Cartão'].str[:1]
df_aba1['emissor_cartao'] = df_aba1['Cartão'].str[1:6]

### 2.3 Filtro

Como as colunas de Hora e timestamp não são mais necessárias dada a redundância:

In [19]:
df_aba1.drop(columns=['Hora', 'transaction_timestamp'], inplace=True)

# 3. Resultados e exportação

In [20]:
df_aba1.head(10)

,Dia,Valor,Cartão,CBK,periodo_do_dia,dia_da_semana,flag_feriado,bandeira_cartao,emissor_cartao
0,2015-05-01,36.54,536518******2108,0,madrugada,Sexta-feira,1,5,36518
1,2015-05-01,36.54,536518******2108,0,madrugada,Sexta-feira,1,5,36518
2,2015-05-01,69.00,453211******1239,0,madrugada,Sexta-feira,1,4,53211
3,2015-05-01,193.43,548827******1705,0,madrugada,Sexta-feira,1,5,48827
4,2015-05-01,132.00,531681******9778,0,madrugada,Sexta-feira,1,5,31681
5,2015-05-01,161.00,515117******4107,0,madrugada,Sexta-feira,1,5,15117
6,2015-05-01,110.00,432032******9111,0,dia,Sexta-feira,1,4,32032
7,2015-05-01,159.50,544540******7141,0,dia,Sexta-feira,1,5,44540
8,2015-05-01,126.50,554906******0358,1,dia,Sexta-feira,1,5,54906
9,2015-05-01,126.50,554906******0358,1,dia,Sexta-feira,1,5,54906


In [21]:
df_aba1.to_csv('../data/processed/card_transactions_cbk_prepared.csv', index=False)